# Lab 3.6.4: Multi-Agent System - SOLUTIONS

**Complete solutions with explanations and alternative approaches**

---

## Setup

In [ ]:
from typing import Dict, Any, List, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
from abc import ABC, abstractmethod
import json
from datetime import datetime
import random

print("Setup complete!")

---

## Exercise 1 Solution: Content Creation Team

**Task**: Build a team of specialized agents that collaborate to create content.

In [ ]:
class AgentRole(Enum):
    RESEARCHER = "researcher"
    WRITER = "writer"
    EDITOR = "editor"
    FACT_CHECKER = "fact_checker"
    SEO_OPTIMIZER = "seo_optimizer"


@dataclass
class Message:
    """Message passed between agents."""
    sender: str
    recipient: str
    content: Any
    message_type: str
    timestamp: float = field(default_factory=lambda: datetime.now().timestamp())
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class AgentOutput:
    """Output from an agent."""
    content: Any
    confidence: float
    suggestions: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)


class BaseAgent(ABC):
    """Abstract base class for agents."""
    
    def __init__(self, name: str, role: AgentRole):
        self.name = name
        self.role = role
        self.message_history: List[Message] = []
    
    @abstractmethod
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Process a task and return output."""
        pass
    
    def receive_message(self, message: Message) -> None:
        """Receive a message from another agent."""
        self.message_history.append(message)
    
    def get_recent_messages(self, count: int = 5) -> List[Message]:
        """Get recent messages."""
        return self.message_history[-count:]


class ResearcherAgent(BaseAgent):
    """Agent specialized in research and information gathering."""
    
    def __init__(self, name: str = "Researcher"):
        super().__init__(name, AgentRole.RESEARCHER)
        self.knowledge_base = {}
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Research a topic and return findings."""
        topic = context.get("topic", task)
        
        # Simulated research
        research = {
            "key_facts": [
                f"Important fact 1 about {topic}",
                f"Key statistic related to {topic}",
                f"Recent development in {topic}",
            ],
            "sources": [
                "Academic Journal (2024)",
                "Industry Report",
                "Expert Interview",
            ],
            "keywords": topic.lower().split()[:5],
            "summary": f"Comprehensive research on {topic} covering key aspects and recent developments.",
        }
        
        self.knowledge_base[topic] = research
        
        return AgentOutput(
            content=research,
            confidence=0.85,
            suggestions=["Consider adding more recent statistics", "Expert quotes would strengthen the content"],
            metadata={"sources_count": 3, "facts_count": 3}
        )


class WriterAgent(BaseAgent):
    """Agent specialized in content writing."""
    
    def __init__(self, name: str = "Writer", style: str = "professional"):
        super().__init__(name, AgentRole.WRITER)
        self.style = style
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Write content based on research."""
        research = context.get("research", {})
        topic = context.get("topic", task)
        word_count = context.get("word_count", 500)
        
        # Extract research findings
        facts = research.get("key_facts", [])
        summary = research.get("summary", "")
        
        # Generate article structure
        article = {
            "title": f"Understanding {topic.title()}: A Comprehensive Guide",
            "introduction": f"In today's rapidly evolving landscape, {topic} has become increasingly important. {summary}",
            "sections": [
                {
                    "heading": "Key Insights",
                    "content": "\n".join([f"- {fact}" for fact in facts])
                },
                {
                    "heading": "Why It Matters",
                    "content": f"Understanding {topic} is crucial for professionals and organizations looking to stay ahead."
                },
                {
                    "heading": "Looking Ahead",
                    "content": f"The future of {topic} promises exciting developments and opportunities."
                },
            ],
            "conclusion": f"As we've explored, {topic} represents a significant area of focus. Stay informed and adaptable to leverage these insights.",
        }
        
        # Estimate word count
        full_text = article["introduction"] + article["conclusion"]
        for section in article["sections"]:
            full_text += section["content"]
        actual_words = len(full_text.split())
        
        return AgentOutput(
            content=article,
            confidence=0.8,
            suggestions=["Add more specific examples", "Include call-to-action"],
            metadata={"word_count": actual_words, "sections": len(article["sections"])}
        )


class EditorAgent(BaseAgent):
    """Agent specialized in editing and improving content."""
    
    def __init__(self, name: str = "Editor"):
        super().__init__(name, AgentRole.EDITOR)
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Edit and improve content."""
        article = context.get("article", {})
        
        # Simulated editing
        edits = {
            "grammar_fixes": 3,
            "style_improvements": 5,
            "clarity_enhancements": 2,
            "structure_changes": 1,
        }
        
        # Add editor notes to article
        edited_article = article.copy()
        edited_article["editor_notes"] = [
            "Improved introduction flow",
            "Enhanced transition between sections",
            "Strengthened conclusion with action items",
        ]
        edited_article["quality_score"] = 0.85
        
        # Improve title
        if "title" in edited_article:
            edited_article["title"] = edited_article["title"].replace("Understanding", "Mastering")
        
        return AgentOutput(
            content=edited_article,
            confidence=0.9,
            suggestions=["Consider A/B testing the title", "Add internal links"],
            metadata={"edits_made": sum(edits.values()), "edit_details": edits}
        )


class ContentTeam:
    """
    Multi-agent team for content creation.
    
    Pipeline:
    1. Researcher gathers information
    2. Writer creates content
    3. Editor refines content
    """
    
    def __init__(self):
        self.researcher = ResearcherAgent()
        self.writer = WriterAgent()
        self.editor = EditorAgent()
        self.message_log: List[Message] = []
    
    def _send_message(self, sender: str, recipient: str, 
                      content: Any, msg_type: str) -> None:
        """Log message between agents."""
        msg = Message(sender, recipient, content, msg_type)
        self.message_log.append(msg)
    
    def create_content(self, topic: str, word_count: int = 500) -> Dict[str, Any]:
        """
        Create content through multi-agent collaboration.
        
        Args:
            topic: The topic to write about
            word_count: Target word count
            
        Returns:
            Final content with metadata
        """
        print(f"\n{'='*60}")
        print(f"CONTENT TEAM: Creating content about '{topic}'")
        print(f"{'='*60}")
        
        # Step 1: Research
        print(f"\n[1] {self.researcher.name} is researching...")
        research_output = self.researcher.process(
            task="research",
            context={"topic": topic}
        )
        self._send_message(self.researcher.name, self.writer.name, 
                          research_output.content, "research_findings")
        print(f"    Found {research_output.metadata['facts_count']} key facts")
        print(f"    Confidence: {research_output.confidence:.0%}")
        
        # Step 2: Write
        print(f"\n[2] {self.writer.name} is writing...")
        writer_output = self.writer.process(
            task="write",
            context={
                "topic": topic,
                "research": research_output.content,
                "word_count": word_count,
            }
        )
        self._send_message(self.writer.name, self.editor.name,
                          writer_output.content, "draft_article")
        print(f"    Created {writer_output.metadata['sections']} sections")
        print(f"    Word count: {writer_output.metadata['word_count']}")
        
        # Step 3: Edit
        print(f"\n[3] {self.editor.name} is editing...")
        editor_output = self.editor.process(
            task="edit",
            context={"article": writer_output.content}
        )
        print(f"    Made {editor_output.metadata['edits_made']} improvements")
        print(f"    Quality score: {editor_output.content.get('quality_score', 0):.0%}")
        
        # Compile final result
        result = {
            "article": editor_output.content,
            "research": research_output.content,
            "pipeline_metadata": {
                "topic": topic,
                "research_confidence": research_output.confidence,
                "writing_confidence": writer_output.confidence,
                "editing_confidence": editor_output.confidence,
                "messages_exchanged": len(self.message_log),
            },
            "suggestions": (
                research_output.suggestions + 
                writer_output.suggestions + 
                editor_output.suggestions
            ),
        }
        
        return result


# Test the content team
print("=" * 60)
print("CONTENT CREATION TEAM SOLUTION")
print("=" * 60)

team = ContentTeam()
result = team.create_content("artificial intelligence in healthcare", word_count=800)

print(f"\n--- Final Article ---")
print(f"Title: {result['article']['title']}")
print(f"\nIntroduction: {result['article']['introduction'][:150]}...")
print(f"\nSections: {len(result['article']['sections'])}")
print(f"\nAll Suggestions:")
for suggestion in result['suggestions']:
    print(f"  - {suggestion}")

---

## Exercise 2 Solution: Feedback Loop with Revisions

**Task**: Implement a multi-agent system with iterative feedback and revision cycles.

In [ ]:
@dataclass
class Feedback:
    """Feedback from reviewer."""
    score: float
    issues: List[str]
    suggestions: List[str]
    requires_revision: bool


class ReviewerAgent(BaseAgent):
    """Agent that reviews and provides feedback."""
    
    def __init__(self, name: str = "Reviewer", min_quality: float = 0.8):
        super().__init__(name, AgentRole.EDITOR)
        self.min_quality = min_quality
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Review content and provide feedback."""
        content = context.get("content", {})
        revision_number = context.get("revision", 0)
        
        # Simulate quality assessment (improves with revisions)
        base_score = 0.6 + (revision_number * 0.15)
        score = min(0.95, base_score + random.uniform(-0.05, 0.05))
        
        # Generate feedback
        issues = []
        suggestions = []
        
        if score < 0.7:
            issues.append("Content lacks depth")
            issues.append("Missing supporting evidence")
            suggestions.append("Add more specific examples")
            suggestions.append("Include data/statistics")
        elif score < 0.8:
            issues.append("Structure could be improved")
            suggestions.append("Strengthen transitions between sections")
        elif score < 0.9:
            suggestions.append("Minor polish needed")
            suggestions.append("Consider adding conclusion summary")
        
        feedback = Feedback(
            score=score,
            issues=issues,
            suggestions=suggestions,
            requires_revision=score < self.min_quality
        )
        
        return AgentOutput(
            content=feedback,
            confidence=0.9,
            metadata={"revision_reviewed": revision_number}
        )


class RevisionAgent(BaseAgent):
    """Agent that revises content based on feedback."""
    
    def __init__(self, name: str = "Reviser"):
        super().__init__(name, AgentRole.WRITER)
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Revise content based on feedback."""
        content = context.get("content", {})
        feedback = context.get("feedback")
        revision_number = context.get("revision", 0)
        
        # Apply revisions based on feedback
        revised = content.copy()
        
        # Address issues
        if feedback and feedback.issues:
            revised["revisions"] = revised.get("revisions", []) + [
                f"Addressed: {issue}" for issue in feedback.issues
            ]
        
        # Apply suggestions
        if feedback and feedback.suggestions:
            revised["improvements"] = revised.get("improvements", []) + [
                f"Applied: {suggestion}" for suggestion in feedback.suggestions[:2]
            ]
        
        revised["revision_number"] = revision_number + 1
        
        return AgentOutput(
            content=revised,
            confidence=0.85,
            metadata={"changes_made": len(feedback.issues if feedback else []) + 2}
        )


class FeedbackLoopTeam:
    """
    Multi-agent team with iterative feedback loops.
    
    Flow:
    1. Create initial content
    2. Review content
    3. If below quality threshold, revise
    4. Repeat until quality met or max iterations
    """
    
    def __init__(self, max_revisions: int = 3, min_quality: float = 0.8):
        self.writer = WriterAgent()
        self.reviewer = ReviewerAgent(min_quality=min_quality)
        self.reviser = RevisionAgent()
        self.max_revisions = max_revisions
        self.min_quality = min_quality
    
    def create_with_feedback(self, topic: str) -> Dict[str, Any]:
        """Create content with iterative feedback."""
        print(f"\n{'='*60}")
        print(f"FEEDBACK LOOP: Creating content about '{topic}'")
        print(f"{'='*60}")
        
        # Initial content
        print(f"\n[Initial] Creating first draft...")
        content_output = self.writer.process(
            task="write",
            context={"topic": topic, "research": {"key_facts": [f"Fact about {topic}"]}}
        )
        current_content = content_output.content
        
        revision_history = []
        revision = 0
        
        while revision < self.max_revisions:
            # Review
            print(f"\n[Review {revision + 1}] Evaluating content...")
            review_output = self.reviewer.process(
                task="review",
                context={"content": current_content, "revision": revision}
            )
            feedback = review_output.content
            
            print(f"    Score: {feedback.score:.2%}")
            print(f"    Issues: {len(feedback.issues)}")
            print(f"    Requires revision: {feedback.requires_revision}")
            
            revision_history.append({
                "revision": revision,
                "score": feedback.score,
                "issues": feedback.issues,
                "suggestions": feedback.suggestions,
            })
            
            if not feedback.requires_revision:
                print(f"\n[Success] Quality threshold met!")
                break
            
            # Revise
            print(f"\n[Revise {revision + 1}] Applying feedback...")
            revision_output = self.reviser.process(
                task="revise",
                context={
                    "content": current_content,
                    "feedback": feedback,
                    "revision": revision,
                }
            )
            current_content = revision_output.content
            print(f"    Changes made: {revision_output.metadata['changes_made']}")
            
            revision += 1
        
        return {
            "final_content": current_content,
            "revision_history": revision_history,
            "total_revisions": revision,
            "final_score": revision_history[-1]["score"] if revision_history else 0,
            "met_quality_threshold": revision_history[-1]["score"] >= self.min_quality if revision_history else False,
        }


# Test feedback loop
print("=" * 60)
print("FEEDBACK LOOP SOLUTION")
print("=" * 60)

feedback_team = FeedbackLoopTeam(max_revisions=3, min_quality=0.8)
result = feedback_team.create_with_feedback("machine learning")

print(f"\n--- Final Results ---")
print(f"Total revisions: {result['total_revisions']}")
print(f"Final score: {result['final_score']:.2%}")
print(f"Met threshold: {result['met_quality_threshold']}")
print(f"\nScore progression:")
for rev in result['revision_history']:
    print(f"  Revision {rev['revision']}: {rev['score']:.2%}")

---

## Exercise 3 Solution: Supervisor-Worker Pattern

**Task**: Build a supervisor agent that routes tasks to specialized workers.

In [ ]:
class TaskType(Enum):
    RESEARCH = "research"
    WRITING = "writing"
    ANALYSIS = "analysis"
    CODING = "coding"
    GENERAL = "general"


@dataclass
class Task:
    """Task to be processed."""
    id: str
    description: str
    task_type: TaskType
    priority: int = 1
    context: Dict[str, Any] = field(default_factory=dict)


class WorkerAgent(BaseAgent):
    """Generic worker agent."""
    
    def __init__(self, name: str, specialization: TaskType):
        super().__init__(name, AgentRole.RESEARCHER)
        self.specialization = specialization
        self.tasks_completed = 0
    
    def can_handle(self, task_type: TaskType) -> bool:
        """Check if worker can handle task type."""
        return task_type == self.specialization or task_type == TaskType.GENERAL
    
    def process(self, task: str, context: Dict[str, Any]) -> AgentOutput:
        """Process assigned task."""
        task_obj = context.get("task")
        
        # Simulated processing
        result = {
            "worker": self.name,
            "specialization": self.specialization.value,
            "task_id": task_obj.id if task_obj else "unknown",
            "output": f"Completed {self.specialization.value} task: {task_obj.description if task_obj else task}",
            "quality": 0.85 + random.uniform(0, 0.1),
        }
        
        self.tasks_completed += 1
        
        return AgentOutput(
            content=result,
            confidence=0.9,
            metadata={"tasks_completed": self.tasks_completed}
        )


class SupervisorAgent:
    """
    Supervisor that routes tasks to appropriate workers.
    
    Responsibilities:
    1. Classify incoming tasks
    2. Route to appropriate worker
    3. Monitor worker performance
    4. Aggregate results
    """
    
    def __init__(self):
        self.workers: Dict[str, WorkerAgent] = {}
        self.task_queue: List[Task] = []
        self.completed_tasks: List[Dict] = []
    
    def add_worker(self, worker: WorkerAgent) -> None:
        """Add a worker to the pool."""
        self.workers[worker.name] = worker
    
    def classify_task(self, description: str) -> TaskType:
        """Classify task based on description."""
        desc_lower = description.lower()
        
        if any(word in desc_lower for word in ["research", "find", "search", "investigate"]):
            return TaskType.RESEARCH
        elif any(word in desc_lower for word in ["write", "create", "draft", "compose"]):
            return TaskType.WRITING
        elif any(word in desc_lower for word in ["analyze", "evaluate", "assess", "compare"]):
            return TaskType.ANALYSIS
        elif any(word in desc_lower for word in ["code", "implement", "program", "debug"]):
            return TaskType.CODING
        else:
            return TaskType.GENERAL
    
    def find_worker(self, task_type: TaskType) -> Optional[WorkerAgent]:
        """Find best available worker for task type."""
        # First, look for specialist
        for worker in self.workers.values():
            if worker.specialization == task_type:
                return worker
        
        # Fall back to any available worker
        for worker in self.workers.values():
            if worker.can_handle(task_type):
                return worker
        
        return None
    
    def submit_task(self, description: str, priority: int = 1) -> str:
        """Submit a new task."""
        task_type = self.classify_task(description)
        task_id = f"TASK_{len(self.task_queue) + len(self.completed_tasks) + 1:03d}"
        
        task = Task(
            id=task_id,
            description=description,
            task_type=task_type,
            priority=priority,
        )
        
        self.task_queue.append(task)
        return task_id
    
    def process_queue(self) -> List[Dict]:
        """Process all tasks in queue."""
        results = []
        
        # Sort by priority
        self.task_queue.sort(key=lambda t: t.priority, reverse=True)
        
        while self.task_queue:
            task = self.task_queue.pop(0)
            
            print(f"\n[Supervisor] Processing {task.id}: {task.description[:50]}...")
            print(f"  Classified as: {task.task_type.value}")
            
            # Find worker
            worker = self.find_worker(task.task_type)
            
            if worker:
                print(f"  Assigned to: {worker.name}")
                output = worker.process(task.description, {"task": task})
                
                result = {
                    "task_id": task.id,
                    "description": task.description,
                    "task_type": task.task_type.value,
                    "worker": worker.name,
                    "output": output.content,
                    "status": "completed",
                }
                print(f"  Quality: {output.content['quality']:.2%}")
            else:
                result = {
                    "task_id": task.id,
                    "description": task.description,
                    "task_type": task.task_type.value,
                    "worker": None,
                    "output": None,
                    "status": "no_worker_available",
                }
                print(f"  ERROR: No worker available for {task.task_type.value}")
            
            results.append(result)
            self.completed_tasks.append(result)
        
        return results
    
    def get_worker_stats(self) -> Dict[str, Any]:
        """Get statistics for all workers."""
        return {
            worker.name: {
                "specialization": worker.specialization.value,
                "tasks_completed": worker.tasks_completed,
            }
            for worker in self.workers.values()
        }


# Test supervisor-worker pattern
print("=" * 60)
print("SUPERVISOR-WORKER PATTERN SOLUTION")
print("=" * 60)

supervisor = SupervisorAgent()

# Add specialized workers
supervisor.add_worker(WorkerAgent("ResearchBot", TaskType.RESEARCH))
supervisor.add_worker(WorkerAgent("WriterBot", TaskType.WRITING))
supervisor.add_worker(WorkerAgent("AnalystBot", TaskType.ANALYSIS))
supervisor.add_worker(WorkerAgent("CoderBot", TaskType.CODING))

# Submit tasks
tasks = [
    "Research the latest trends in renewable energy",
    "Write a blog post about machine learning",
    "Analyze the performance metrics from Q4",
    "Implement a sorting algorithm in Python",
    "Find information about climate change policies",
]

print("\n--- Submitting Tasks ---")
for task_desc in tasks:
    task_id = supervisor.submit_task(task_desc)
    print(f"Submitted: {task_id}")

print("\n--- Processing Queue ---")
results = supervisor.process_queue()

print(f"\n--- Worker Statistics ---")
for name, stats in supervisor.get_worker_stats().items():
    print(f"{name}: {stats['tasks_completed']} tasks ({stats['specialization']})")

---

## Challenge Solution: Debate System

**Task**: Create a multi-agent debate system where agents argue different positions and reach consensus.

In [ ]:
@dataclass
class Argument:
    """An argument in the debate."""
    agent: str
    position: str
    points: List[str]
    strength: float
    round_number: int


class DebateAgent:
    """Agent that argues a position in a debate."""
    
    def __init__(self, name: str, position: str):
        self.name = name
        self.position = position
        self.arguments: List[Argument] = []
        self.concessions: List[str] = []
    
    def make_argument(self, topic: str, round_num: int, 
                      opponent_args: List[Argument]) -> Argument:
        """Generate an argument for the current round."""
        # Generate points based on position
        if self.position == "pro":
            points = [
                f"Supporting evidence for {topic}",
                f"Positive impact on stakeholders",
                f"Historical success of similar approaches",
            ]
        else:
            points = [
                f"Concerns regarding {topic}",
                f"Potential risks and challenges",
                f"Alternative approaches to consider",
            ]
        
        # Respond to opponent arguments
        if opponent_args:
            latest_opponent = opponent_args[-1]
            points.append(f"Rebuttal to '{latest_opponent.points[0][:30]}...'")
        
        # Calculate argument strength (improves with rounds as debate refines)
        strength = 0.6 + (round_num * 0.1) + random.uniform(-0.05, 0.1)
        strength = min(0.95, strength)
        
        argument = Argument(
            agent=self.name,
            position=self.position,
            points=points,
            strength=strength,
            round_number=round_num,
        )
        
        self.arguments.append(argument)
        return argument
    
    def consider_concession(self, opponent_arg: Argument) -> Optional[str]:
        """Consider whether to concede a point."""
        # Concede if opponent argument is strong
        if opponent_arg.strength > 0.85 and random.random() > 0.5:
            concession = f"Acknowledging valid point: {opponent_arg.points[0][:50]}"
            self.concessions.append(concession)
            return concession
        return None


class ModeratorAgent:
    """Moderator that guides debate and determines consensus."""
    
    def __init__(self, name: str = "Moderator"):
        self.name = name
        self.rounds: List[Dict] = []
    
    def evaluate_round(self, pro_arg: Argument, con_arg: Argument) -> Dict:
        """Evaluate a debate round."""
        round_result = {
            "round": pro_arg.round_number,
            "pro_strength": pro_arg.strength,
            "con_strength": con_arg.strength,
            "round_winner": "pro" if pro_arg.strength > con_arg.strength else "con",
            "margin": abs(pro_arg.strength - con_arg.strength),
        }
        
        self.rounds.append(round_result)
        return round_result
    
    def determine_consensus(self) -> Dict:
        """Determine debate outcome and consensus."""
        if not self.rounds:
            return {"error": "No rounds to evaluate"}
        
        pro_wins = sum(1 for r in self.rounds if r["round_winner"] == "pro")
        con_wins = len(self.rounds) - pro_wins
        
        avg_margin = sum(r["margin"] for r in self.rounds) / len(self.rounds)
        
        # Determine outcome
        if pro_wins > con_wins and avg_margin > 0.1:
            outcome = "pro_victory"
            recommendation = "Support the proposition"
        elif con_wins > pro_wins and avg_margin > 0.1:
            outcome = "con_victory"
            recommendation = "Oppose the proposition"
        else:
            outcome = "consensus_needed"
            recommendation = "Seek middle ground incorporating valid points from both sides"
        
        return {
            "outcome": outcome,
            "pro_rounds_won": pro_wins,
            "con_rounds_won": con_wins,
            "average_margin": avg_margin,
            "recommendation": recommendation,
            "confidence": 0.7 + avg_margin,
        }


class DebateSystem:
    """
    Multi-agent debate system.
    
    Flow:
    1. Initialize pro and con agents
    2. Conduct multiple rounds of debate
    3. Each round: arguments, rebuttals, potential concessions
    4. Moderator evaluates and determines consensus
    """
    
    def __init__(self, max_rounds: int = 3):
        self.max_rounds = max_rounds
        self.moderator = ModeratorAgent()
    
    def conduct_debate(self, topic: str) -> Dict:
        """Conduct a full debate on a topic."""
        print(f"\n{'='*60}")
        print(f"DEBATE: {topic}")
        print(f"{'='*60}")
        
        # Initialize debaters
        pro_agent = DebateAgent("ProAgent", "pro")
        con_agent = DebateAgent("ConAgent", "con")
        
        all_arguments = []
        
        for round_num in range(1, self.max_rounds + 1):
            print(f"\n--- Round {round_num} ---")
            
            # Pro argument
            pro_args = [a for a in all_arguments if a.position == "con"]
            pro_argument = pro_agent.make_argument(topic, round_num, pro_args)
            all_arguments.append(pro_argument)
            print(f"\n[PRO] {pro_agent.name} (strength: {pro_argument.strength:.2f}):")
            for point in pro_argument.points[:2]:
                print(f"  - {point}")
            
            # Con argument
            con_args = [a for a in all_arguments if a.position == "pro"]
            con_argument = con_agent.make_argument(topic, round_num, con_args)
            all_arguments.append(con_argument)
            print(f"\n[CON] {con_agent.name} (strength: {con_argument.strength:.2f}):")
            for point in con_argument.points[:2]:
                print(f"  - {point}")
            
            # Check for concessions
            pro_concession = pro_agent.consider_concession(con_argument)
            con_concession = con_agent.consider_concession(pro_argument)
            
            if pro_concession:
                print(f"\n[PRO CONCEDES] {pro_concession}")
            if con_concession:
                print(f"\n[CON CONCEDES] {con_concession}")
            
            # Moderator evaluation
            round_result = self.moderator.evaluate_round(pro_argument, con_argument)
            print(f"\n[MODERATOR] Round winner: {round_result['round_winner'].upper()} (margin: {round_result['margin']:.2f})")
        
        # Final consensus
        consensus = self.moderator.determine_consensus()
        
        print(f"\n{'='*60}")
        print("DEBATE CONCLUSION")
        print(f"{'='*60}")
        print(f"Outcome: {consensus['outcome']}")
        print(f"Pro rounds won: {consensus['pro_rounds_won']}")
        print(f"Con rounds won: {consensus['con_rounds_won']}")
        print(f"Recommendation: {consensus['recommendation']}")
        print(f"Confidence: {consensus['confidence']:.2%}")
        
        return {
            "topic": topic,
            "arguments": all_arguments,
            "pro_concessions": pro_agent.concessions,
            "con_concessions": con_agent.concessions,
            "consensus": consensus,
        }


# Test debate system
print("=" * 60)
print("DEBATE SYSTEM SOLUTION")
print("=" * 60)

debate = DebateSystem(max_rounds=3)
result = debate.conduct_debate("AI should be regulated by governments")

---

## Key Takeaways

1. **Message Passing**: Agents communicate through structured messages
2. **Role Specialization**: Each agent has specific capabilities and responsibilities
3. **Feedback Loops**: Iterative refinement improves output quality
4. **Supervisor Pattern**: Central coordination enables complex task routing
5. **Consensus Building**: Multiple perspectives lead to better decisions